In [9]:
!pip install pandas_ta

Defaulting to user installation because normal site-packages is not writeable


In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import json
import requests
import pandas_ta as ta

2019-04-15: start

In [127]:


def get_dataframe(ticker, start, end):
    response = requests.get('https://api.senticrypt.com/v2/all.json')
    if response.status_code == 200:
        data = response.json()
        print(data)
    else:
        print(f"Error: {response.status_code}")


    df = pd.DataFrame(data)
    df = df.iloc[::-1]
    df['Date'] = pd.to_datetime(df['date'])
    df.drop(columns=['date'], inplace = True)
    
    

    
    data1 = yf.download(tickers = ticker, start = start,end = end)
    data = pd.merge(data1, df, on = 'Date')
    data.drop(columns=['price','volume','score1','score2','score3','count',	'sum'], inplace = True)
    data['RSI']=ta.rsi(data.Close, length=15)
    data['EMAF']=ta.ema(data.Close, length=20)
    data['EMAM']=ta.ema(data.Close, length=100)
    data['EMAS']=ta.ema(data.Close, length=150)

    data['Target'] = data['Adj Close']-data.Open
    data['Target'] = data['Target'].shift(-1)

    data['TargetClass'] = [1 if data.Target[i]>0 else 0 for i in range(len(data))]

    data['TargetNextClose'] = data['Adj Close'].shift(-1)

    data.dropna(inplace=True)
    data.reset_index(inplace = True)
    data.drop(['Volume', 'Close','Date','index'], axis=1, inplace=True)
    data_set = data.iloc[:, 0:12]
    pd.set_option('display.max_columns', None)
    
    return data_set

In [128]:
result = get_dataframe('BTC-USD', '2019-04-15', str(pd.Timestamp.today()).split(' ')[0])

[*********************100%%**********************]  1 of 1 completed

[{'date': '2024-03-07', 'price': 69324.0, 'volume': 41012.0, 'score1': -0.3201, 'score2': 0.1029, 'score3': 0.0514, 'count': 24, 'mean': -0.0553, 'sum': -1.3264}, {'date': '2024-03-06', 'price': 69324.0, 'volume': 41012.0, 'score1': -0.3023, 'score2': 0.0826, 'score3': 0.1705, 'count': 78, 'mean': -0.0164, 'sum': -1.2808}, {'date': '2024-03-05', 'price': 62655.0, 'volume': 3543.0, 'score1': -0.264, 'score2': 0.1082, 'score3': 0.1373, 'count': 168, 'mean': -0.0062, 'sum': -1.0376}, {'date': '2024-03-04', 'price': 62655.0, 'volume': 3543.0, 'score1': -0.3095, 'score2': 0.1296, 'score3': 0.1804, 'count': 180, 'mean': 0.0002, 'sum': 0.0373}, {'date': '2024-03-03', 'price': 62500.0, 'volume': 3780.0, 'score1': -0.1639, 'score2': 0.1705, 'score3': 0.2366, 'count': 186, 'mean': 0.0811, 'sum': 15.0793}, {'date': '2024-03-02', 'price': 63675.0, 'volume': 9980.0, 'score1': -0.0654, 'score2': 0.215, 'score3': 0.2016, 'count': 183, 'mean': 0.117, 'sum': 21.4192}, {'date': '2024-03-01', 'price': 58

In [130]:
result.columns

Index(['Open', 'High', 'Low', 'Adj Close', 'mean', 'RSI', 'EMAF', 'EMAM',
       'EMAS', 'Target', 'TargetClass', 'TargetNextClose'],
      dtype='object')

In [131]:
result.to_csv('btc_data.csv', index=False)